In [ ]:
Feb-March-April
Sept-Oct-Nov

In [30]:
import pygmo as pg
import pandas as pd
import numpy as np

from IPython.display import clear_output, display
import sys
sys.path.append('../../../xray/src/problems/DIVAR3V/library')
# sys.path.append('~/git/xray/src/problems/DIVAR3V/library')
from sim_funcs_CMA import *

In [33]:
# Setting up a problem
class DIVAR3V:
    def __init__(self):
        self.input_path = '~/git/research/xray/src/problems/DIVAR3V/input/'
#         self.output_path = '~/git/research/DIVA-R1.1.3-V/src/CMA-ES/urdfs/'
        self.output_path = './urdfs/'
        self.problem_dv = pd.read_csv(self.input_path+'dv_space.csv')
        self.problem_qoi = pd.read_csv(self.input_path+'qoi_space.csv')
        self.dim = len(self.problem_dv.Variables)
    
    def fitness(self,sample_dv):
        # Print latest sample being evaluated
#         clear_output(wait=True)
#         print(sample_dv)
        # workspace, applied_force, payload, ts, total_friction_loss, total_joule_loss
        results = np.array(compute_bottom_up_mappings(sample_dv, path = self.output_path))
        # Handle each of the cases
        # if any of the workspace, applied force, payload and ts are -1
        if np.any(results[:4]==-1):
            # Then set them as very bad designs by setting the corresponding cost too high
            results[:4] = 1e7
            
        # Add weighting factor, K_w
        # workspace, applied_force, payload, ts, total_friction_loss, total_joule_loss
        K_w = np.diag([100,100,100,100,100,1])
        fitness_val = np.matmul(K_w, results).dot(results)
        fitness_val = results.sum()
        return [fitness_val]
    
    def get_bounds(self):
        dv_bounds = np.vstack([self.problem_dv.Lower, self.problem_dv.Upper]).transpose()
        dv_bounds_tuple = tuple(dv_bounds.transpose().tolist())
        dv_bounds_tuple[0][-2] = 8
        return dv_bounds_tuple
    
    def get_name(self):
        return "DIVAR3V"

In [34]:
prob = pg.problem(DIVAR3V())
# algo = pg.algorithm(pg.bee_colony(gen = 2, limit = 2))
# algo.set_verbosity(100)
print(prob)
print(prob.get_bounds())

Problem name: DIVAR3V
	C++ class name: pybind11::object

	Global dimension:			12
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [0.1, 0.1, 0.05, 0.1, 0.05, ... ]
	Upper bounds: [0.5, 0.5, 0.95, 0.5, 0.95, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none

(array([ 0.1 ,  0.1 ,  0.05,  0.1 ,  0.05,  1.  ,  8.  ,  0.1 ,  1.  ,
       40.  ,  8.  ,  0.1 ]), array([  0.5 ,   0.5 ,   0.95,   0.5 ,   0.95,  20.  ,  40.  ,   1.  ,
        40.  , 200.  ,  20.  ,   0.25]))


In [37]:
# Using CMA-ES
algo = pg.algorithm(pg.cmaes(gen = 70, sigma0=0.3))
print(algo)
algo.set_verbosity(200)

Algorithm name: CMA-ES: Covariance Matrix Adaptation Evolutionary Strategy [stochastic]
	C++ class name: pagmo::cmaes

	Thread safety: basic

Extra info:
	Generations: 70
	cc: auto
	cs: auto
	c1: auto
	cmu: auto
	sigma0: 0.3
	Stopping xtol: 1e-06
	Stopping ftol: 1e-06
	Memory: false
	Verbosity: 0
	Force bounds: false
	Seed: 305893601


In [ ]:
archi = pg.archipelago(n=16, algo = algo, prob=prob, pop_size=100)
archi.evolve(); archi.wait()


   Gen:        Fevals:          Best:            dx:            df:         sigma:
      1              0        6.69082         36.134          4e+07            0.3
CMAES 4 PaGMO: 
mu: 50 - lambda: 100 - mueff: 26.9667 - N: 12
cc: 0.304825 - cs: 0.658832 - c1: 0.00981082 - cmu: 0.224282 - sigma: 0.3 - damps: 2.48545 - chiN: 3.39308

   Gen:        Fevals:          Best:            dx:            df:         sigma:
      1              0        7.67956         36.134          4e+07            0.3
CMAES 4 PaGMO: 
mu: 50 - lambda: 100 - mueff: 26.9667 - N: 12
cc: 0.304825 - cs: 0.658832 - c1: 0.00981082 - cmu: 0.224282 - sigma: 0.3 - damps: 2.48545 - chiN: 3.39308

   Gen:        Fevals:          Best:            dx:            df:         sigma:
      1              0        6.82666         36.134          4e+07            0.3
CMAES 4 PaGMO: 
mu: 50 - lambda: 100 - mueff: 26.9667 - N: 12
cc: 0.304825 - cs: 0.658832 - c1: 0.00981082 - cmu: 0.224282 - sigma: 0.3 - damps: 2.48545 - chiN: 

In [ ]:
a = archi.get_champions_f()
a2 = sorted(archi.get_champions_f(), key = lambda x: x[0])[0]
best_isl_idx = [(el == a2).all() for el in a].index(True)
x_best = archi.get_champions_x()[best_isl_idx]
f_best = archi.get_champions_f()[best_isl_idx]
print("Best relaxed solution, x: {}".format(x_best)) 
print("Best relaxed solution, f: {}".format(f_best)) 